In [ ]:
from typing import Tuple
import geopandas as gpd
import momepy
import momepy.coins
import numpy as np
import pandas as pd
import sklearn.cluster
import shapely

In [ ]:
# check it is the latest momepy
momepy.__version__

# Corridor Segments

Load the street network (edges only), the waterway, and the corridor: 

In [3]:
city_name = "Bucharest"
river_name = "Dâmbovița"

In [4]:
edges = gpd.read_file(f"../../data/generated/network_edges_{city_name}.gpkg")
waterway = gpd.read_file(f"../../data/generated/waterway_{river_name}.gpkg")
corridor = gpd.read_file(f"../../data/generated/corridor_{river_name}.gpkg")

Extract the relevant geometries:

In [5]:
waterway_geometry = waterway.iloc[0].geometry
corridor_geometry = corridor.iloc[0].geometry
edge1_geometry = corridor.iloc[1].geometry
edge2_geometry = corridor.iloc[2].geometry

## Continuity analysis (modified `momepy`)

In [6]:
def _cross_check_links(unique, angle_pairs, angle_threshold):
    """
    Modified version of the momepy function that identifies "best"
    links between segments, dropping the reciprocity requirement.
    """
    for edge in range(0, len(unique)):
        best_p1 = unique[edge][4][0]
        best_p2 = unique[edge][5][0]

        if (
            isinstance(best_p1, int)
            # and edge in [unique[best_p1][4][0], unique[best_p1][5][0]]
            and angle_pairs["%d_%d" % (edge, best_p1)] > angle_threshold
        ):
            unique[edge][6] = best_p1
        else:
            unique[edge][6] = "line_break"

        if (
            isinstance(best_p2, int)
            # and edge in [unique[best_p2][4][0], unique[best_p2][5][0]]
            and angle_pairs["%d_%d" % (edge, best_p2)] > angle_threshold
        ):
            unique[edge][7] = best_p2
        else:
            unique[edge][7] = "line_break"

In [7]:
def _merge_lines_loop(n, unique_dict, bound_geometry):
    """
    Modified version of the momepy function that merges the line
    segments:
    - crossing a provided boundary geometry is added as stopping condition;
    - when assigning the best continuation segment on a vertex, we force the
      following iteration to continue on the other vertex of the segment.
      This is probably needed as a consequence of having dropped the reciprocity
      requirement in assigning the best links.
    """
    outlist = set()
    current_edge1 = n
    vertex = None

    outlist.add(current_edge1)
    while True:
        p1, p2 = [tuple(i) for i in unique_dict[current_edge1][0]]
        edge = shapely.LineString([p1, p2])
        if not bound_geometry.contains(edge):
            break
        if (
            isinstance(unique_dict[current_edge1][6], int)
            and unique_dict[current_edge1][6] not in outlist
            and ((vertex is None) or (vertex == p2))
        ):
            current_edge1 = unique_dict[current_edge1][6]
            outlist.add(current_edge1)
            vertex = tuple(p1)
        elif (
            isinstance(unique_dict[current_edge1][7], int)
            and unique_dict[current_edge1][7] not in outlist
            and ((vertex is None) or (vertex == p1))
        ):
            current_edge1 = unique_dict[current_edge1][7]
            outlist.add(current_edge1)
            vertex = tuple(p2)
        else:
            break

    current_edge1 = n
    vertex = None
    while True:
        p1, p2 = [tuple(i) for i in unique_dict[current_edge1][0]]
        edge = shapely.LineString([p1, p2])
        if not bound_geometry.contains(edge):
            break
        if (
            isinstance(unique_dict[current_edge1][7], int)
            and unique_dict[current_edge1][7] not in outlist
            and ((vertex is None) or (vertex == p1))
        ):
            current_edge1 = unique_dict[current_edge1][7]
            outlist.add(current_edge1)
            vertex = tuple(p2)
        elif (
            isinstance(unique_dict[current_edge1][6], int)
            and unique_dict[current_edge1][6] not in outlist
            and ((vertex is None) or (vertex == p2))
        ):
            current_edge1 = unique_dict[current_edge1][6]
            outlist.add(current_edge1)
            vertex = tuple(p1)
        else:
            break

    outlist = list(outlist)
    outlist.sort()
    return outlist


In [ ]:
edges_simplified = momepy.roundabout_simplification(edges)

In [ ]:
# show effect of roundabout simplification
m = edges.explore(color="red")
m = edges_simplified.explore(m=m, color="black")
m

In [10]:
# make continuity analysis using COINS
ANGLE_THRESHOLD = 100.
coins = momepy.coins.COINS(edges, angle_threshold=ANGLE_THRESHOLD, flow_mode=True)
# get GDF of all line segments
premerge = coins._create_gdf_premerge()
# find the segments intersecting the waterway
mask = premerge.intersects(waterway_geometry)
crossings = premerge[mask]

In [11]:
# modified assignment of best continuation edges (with threshold)
_cross_check_links(coins.unique, coins.angle_pairs, ANGLE_THRESHOLD)

In [12]:
# find the crossing points as intersections between the waterway and the street network
crossing_points = crossings.intersection(waterway_geometry) # this should now be "point" geometries

In [ ]:
# grow strokes from the crossings
geoms = []
for idx in crossings.index:
    indices = _merge_lines_loop(idx, coins.unique, corridor_geometry)
    geoms.append(premerge.loc[indices].unary_union)
strokes = gpd.GeoDataFrame(geometry=geoms, crs=crossings.crs)

## Use the strokes to "cut" the corridor

In [14]:
# use the crossing points to define clusters
xy = np.column_stack([crossing_points.x, crossing_points.y])
dbscan = sklearn.cluster.DBSCAN(eps=100, min_samples=1)
dbscan.fit(xy)

# assign cluster labels to the strokes
strokes["cluster"] = dbscan.labels_

In [15]:
# find the strokes that intersect both edges of the corridor
mask = strokes.intersects(edge1_geometry) & strokes.intersects(edge2_geometry)
filtered = strokes[mask]

In [16]:
# for each cluster, select the shortest strokes
filtered = filtered.assign(length=filtered.length)
idx = filtered.groupby("cluster").agg({"length": "idxmin"})
shortest = filtered.loc[idx["length"].values]

In [ ]:
# explore the strokes identified
m = corridor.explore()
m = shortest.explore(m=m, color="red")
m = crossings.explore(m=m, color="green")
# m.save("output.html")
m

In [18]:
def cut_into_blocks(corridor: shapely.Polygon, edges: gpd.GeoSeries) -> gpd.GeoSeries:
    """
    Cut a polygon into blocks, using the provided linestrings as edges.
    """
    lines = edges.explode().to_list()
    lines.append(corridor.boundary)
    lines_merged = shapely.ops.linemerge(lines)
    border_lines = shapely.ops.unary_union(lines_merged)
    decomposition = shapely.ops.polygonize(border_lines)

    # decomposition can extend beyond the corridor - clip it now
    decomposition_gdf = gpd.GeoSeries(decomposition, crs=edges.crs)
    clipped = decomposition_gdf.clip(corridor)

    # drop elements at the edges (LineStrings and Points)
    blocks = clipped[clipped.type == "Polygon"]
    return blocks.reset_index(drop=True)

In [19]:
# cut corridor into blocks
blocks = cut_into_blocks(corridor_geometry, shortest.geometry)

## Merge blocks into river segments

In [20]:
def merge_blocks(blocks: gpd.GeoSeries, mask: pd.Series) -> Tuple[gpd.GeoSeries]:
    """
    Merge a selection of blocks with the smallest adjacent ones.
    Run this recursively, considering the blocks specified with a mask
    from the smallest to the largest.
    """
    # sort blocks (and mask) according to block areas
    idx_sorted = blocks.area.sort_values().index
    blocks = blocks[idx_sorted]
    mask = mask[idx_sorted]

    # get index of the first block to consider (i.e. the smallest)
    idx = mask.idxmax()
    if idx:
        # extract block from table
        block = blocks[idx]
        blocks = blocks.drop(idx)
        mask = mask.drop(idx)
        # find adjacent blocks (i.e. blocks intersecting the target
        # by a linestring)
        intersection_type = blocks.intersection(block).geom_type
        is_adjacent = intersection_type.str.contains("LineString")
        # get index of the first (i.e. smallest) adjacent block
        idx_to_merge = is_adjacent.idxmax()
        if not idx_to_merge:
            raise ValueError(
                f"Block {idx_to_merge} does not have valid (LineString) "
                f"instersections with other blocks."
            )
        # merge geometries
        blocks.loc[idx_to_merge] = blocks[idx_to_merge].union(block)

    if mask.any():
        # recursively merge other blocks, if needed
        blocks, mask = merge_blocks(blocks, mask)
    return blocks, mask

In [21]:
# merge blocks that do not touch the waterway
mask = ~blocks.intersects(waterway_geometry)
blocks, _ = merge_blocks(blocks, mask)

In [22]:
# merge blocks that do not extend across the corridor
mask = ~(blocks.intersects(edge1_geometry) & blocks.intersects(edge2_geometry))
segments, _ = merge_blocks(blocks, mask)

## Visualize result and save output

In [ ]:
# visualize resulting segments
segments.explore()

In [25]:
# save output
segments.to_file(f"../../data/generated/segments_{river_name}.gpkg")